In [1]:
def mostrar_descontos_mensais(funcao):
    salarios = [1_903.98, 2_826.66, 3_751.06, 4_664.68, 8_333.34, 16_666.67, 25_000, 83_333.34, 833_333.34]
    return [(f'R$ {funcao(salario):,.2f}', f'{funcao(salario) / salario:.2%}') for salario in salarios]

In [2]:
def imposto_de_renda_mensal(salario_mensal:float) -> float:
    """
    Calcula imposto de renda PF de acordo com faixa do ano de 2019
    https://impostoderenda2019.net.br/tabela-imposto-de-renda-2019/
    
    """
    saldo = salario_mensal
    aliquotas_faixas = {0.275: 4_664.68, 0.225: 3_751.06, 0.15: 2_826.66, 0.075: 1_903.98, 0: 0}
    ir = 0
    desconto_simplificado = 0.20  # 20% de desconto simplificado
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        ir += delta * aliquota * (1 - desconto_simplificado)
        saldo = min(saldo, faixa)
    return ir


In [3]:
mostrar_descontos_mensais(imposto_de_renda_mensal)


[('R$ 0.00', '0.00%'),
 ('R$ 55.36', '1.96%'),
 ('R$ 166.29', '4.43%'),
 ('R$ 330.74', '7.09%'),
 ('R$ 1,137.85', '13.65%'),
 ('R$ 2,971.18', '17.83%'),
 ('R$ 4,804.51', '19.22%'),
 ('R$ 17,637.85', '21.17%'),
 ('R$ 182,637.85', '21.92%')]

In [4]:
def fgts_empregador(salario_mensal: float)-> float:
    """
    Calcula fgts a ser pago de imposto pelo empregador em 1 ano. 
    Aliquota de 8% + 40% disso em caso de demissão com justa causa -> 11,2%
    
    
    :param salario_mensal: salário mensal bruto do empregado
    :return: float
    """
    return salario_mensal * 0.112


In [5]:
mostrar_descontos_mensais(fgts_empregador)

[('R$ 213.25', '11.20%'),
 ('R$ 316.59', '11.20%'),
 ('R$ 420.12', '11.20%'),
 ('R$ 522.44', '11.20%'),
 ('R$ 933.33', '11.20%'),
 ('R$ 1,866.67', '11.20%'),
 ('R$ 2,800.00', '11.20%'),
 ('R$ 9,333.33', '11.20%'),
 ('R$ 93,333.33', '11.20%')]

In [6]:
def inss_empregado(salario_mensal):
    """
    Calcula inss para 2019: 1.751,81
    :param salario_mensal: 
    :return: 
    """
    teto = 5_839.45
    if salario_mensal < 1_751.81:
        return salario_mensal * 0.08
    elif salario_mensal < 2_919.72:
        return salario_mensal * 0.09
   
    return min(salario_mensal, teto) * 0.11


In [7]:

mostrar_descontos_mensais(inss_empregado)

[('R$ 171.36', '9.00%'),
 ('R$ 254.40', '9.00%'),
 ('R$ 412.62', '11.00%'),
 ('R$ 513.11', '11.00%'),
 ('R$ 642.34', '7.71%'),
 ('R$ 642.34', '3.85%'),
 ('R$ 642.34', '2.57%'),
 ('R$ 642.34', '0.77%'),
 ('R$ 642.34', '0.08%')]

In [8]:
def imposto_de_renda_anual(salario):
    """
    Calcula imposto de renda anual considerando Décimo terceiro e terço de ferias
    :param salario: 
    :return: 
    """
    ir = imposto_de_renda_mensal(salario) * 11
    salario_de_ferias = salario + salario / 3
    ir += imposto_de_renda_mensal(salario_de_ferias - inss_empregado(salario_de_ferias))
    ir += imposto_de_renda_mensal(salario - inss_empregado(salario)) / 0.75  # Tributação exlusiva de décimo terceiro
    return ir


In [9]:
def mostrar_descontos_anuais(funcao):
    salarios = [1_903.98, 2_826.66, 3_751.06, 4_664.68, 8_333.34, 16_666.67, 25_000, 83_333.34, 833_333.34]
    salarios_anuais = [s * 13 + s / 3 for s in salarios]
    for tpl in [
        (f'Salário Mensal R$ {salario:,.2f}', f' Salário Anual R$ {salario_anual:,.2f}', f'R$ {funcao(salario):,.2f}', f'{funcao(salario) / salario_anual:.2%}') for
        salario,  salario_anual in zip(salarios, salarios_anuais)]:
        print(*tpl, sep=' - ')


In [10]:
mostrar_descontos_anuais(imposto_de_renda_anual)

Salário Mensal R$ 1,903.98 -  Salário Anual R$ 25,386.40 - R$ 24.37 - 0.10%
Salário Mensal R$ 2,826.66 -  Salário Anual R$ 37,688.80 - R$ 781.11 - 2.07%
Salário Mensal R$ 3,751.06 -  Salário Anual R$ 50,014.13 - R$ 2,277.20 - 4.55%
Salário Mensal R$ 4,664.68 -  Salário Anual R$ 62,195.73 - R$ 4,487.49 - 7.22%
Salário Mensal R$ 8,333.34 -  Salário Anual R$ 111,111.20 - R$ 15,452.65 - 13.91%
Salário Mensal R$ 16,666.67 -  Salário Anual R$ 222,222.27 - R$ 40,508.20 - 18.23%
Salário Mensal R$ 25,000.00 -  Salário Anual R$ 333,333.33 - R$ 65,563.74 - 19.67%
Salário Mensal R$ 83,333.34 -  Salário Anual R$ 1,111,111.20 - R$ 240,952.65 - 21.69%
Salário Mensal R$ 833,333.34 -  Salário Anual R$ 11,111,111.20 - R$ 2,495,952.65 - 22.46%


In [11]:
def inss_anual(salario):
    """
    Calcula valor total decontado do empregado anualmente
    :param salario: 
    :return: 
    """
    ONZE_MESES_MAIS_DECIMO_TERCEIRO = 12
    inss = inss_empregado(salario) * ONZE_MESES_MAIS_DECIMO_TERCEIRO
    salario_ferias = salario + salario / 3
    return inss + inss_empregado(salario_ferias)


In [12]:
mostrar_descontos_anuais(inss_anual)

Salário Mensal R$ 1,903.98 -  Salário Anual R$ 25,386.40 - R$ 2,284.78 - 9.00%
Salário Mensal R$ 2,826.66 -  Salário Anual R$ 37,688.80 - R$ 3,467.37 - 9.20%
Salário Mensal R$ 3,751.06 -  Salário Anual R$ 50,014.13 - R$ 5,501.55 - 11.00%
Salário Mensal R$ 4,664.68 -  Salário Anual R$ 62,195.73 - R$ 6,799.72 - 10.93%
Salário Mensal R$ 8,333.34 -  Salário Anual R$ 111,111.20 - R$ 8,350.41 - 7.52%
Salário Mensal R$ 16,666.67 -  Salário Anual R$ 222,222.27 - R$ 8,350.41 - 3.76%
Salário Mensal R$ 25,000.00 -  Salário Anual R$ 333,333.33 - R$ 8,350.41 - 2.51%
Salário Mensal R$ 83,333.34 -  Salário Anual R$ 1,111,111.20 - R$ 8,350.41 - 0.75%
Salário Mensal R$ 833,333.34 -  Salário Anual R$ 11,111,111.20 - R$ 8,350.41 - 0.08%


In [13]:
def impostos_anuais(salario):
    return imposto_de_renda_anual(salario)+inss_empregado(salario)

In [14]:
def resumo_empregado(salario):
    """
    Retorna tupla com (Salario Mensal, Salario Mensal liquido,Salario Anual Liquido, Salario Liquido +fgts,Salario Bruto Anual , Impostos, FGTS, 
    :param funcao: 
    :return: 
    """

    salario_anual_bruto = salario * 13 + salario / 3
    impostos = impostos_anuais(salario)
    fgts = fgts_empregador(salario)
    fgts = fgts * 13 + fgts / 3
    salario_mensal_liquido = salario - imposto_de_renda_mensal(salario) - fgts_empregador(salario) - inss_empregado(salario)
    return salario, salario_mensal_liquido, salario_anual_bruto - impostos, salario_anual_bruto - impostos + fgts, salario_anual_bruto, impostos, fgts,


In [15]:
salario_anual_bruto=[1_903.98, 2_826.66, 3_751.06, 4_664.68, 8_333.34, 16_666.67, 25_000, 83_333.34, 833_333.34]

In [16]:
for salario in salario_anual_bruto:
    print(*map(lambda v: f'{v:,.2f}', resumo_empregado(salario)), sep=' - ')

1,903.98 - 1,519.38 - 25,190.67 - 28,033.95 - 25,386.40 - 195.73 - 2,843.28
2,826.66 - 2,200.31 - 36,653.29 - 40,874.44 - 37,688.80 - 1,035.51 - 4,221.15
3,751.06 - 2,752.04 - 47,324.32 - 52,925.90 - 50,014.13 - 2,689.82 - 5,601.58
4,664.68 - 3,298.38 - 57,195.13 - 64,161.05 - 62,195.73 - 5,000.60 - 6,965.92
8,333.34 - 5,619.82 - 95,016.21 - 107,460.66 - 111,111.20 - 16,094.99 - 12,444.45
16,666.67 - 11,186.49 - 181,071.73 - 205,960.62 - 222,222.27 - 41,150.54 - 24,888.89
25,000.00 - 16,753.15 - 267,127.25 - 304,460.58 - 333,333.33 - 66,206.08 - 37,333.33
83,333.34 - 55,719.82 - 869,516.21 - 993,960.66 - 1,111,111.20 - 241,594.99 - 124,444.45
833,333.34 - 556,719.82 - 8,614,516.21 - 9,858,960.66 - 11,111,111.20 - 2,496,594.99 - 1,244,444.45


In [17]:
faturamento_bruto = [13 * s + s / 3 + fgts_empregador(s) for s in salario_anual_bruto]



In [18]:
faturamento_bruto

[25599.645760000003,
 38005.38592,
 50434.25205333333,
 62718.177493333336,
 112044.53408,
 224088.93370666663,
 336133.3333333333,
 1120444.53408,
 11204444.534079999]

In [19]:
def custos_simples_anexo_iii(faturamento_anual, contador=89, prolabore=1_100) -> float:
    """
    Calcula custos de impostos para empresa simples de anexo III
    DAS, Prólabore e Contador
    
    Aliquotas 2019: https://www.contabilizei.com.br/contabilidade-online/anexo-3-simples-nacional/
    :param faturamento_anual: 
    :param contador: 
    :param prolabore: 
    :return: 
    """
    saldo = faturamento_anual
    aliquotas_faixas = {0.33: 3_600_000, 0.21: 1_800_000, 0.16: 720_000, 0.135: 360_000, 0.112: 180_000, 0.06: 0}
    imposto = 0
    for aliquota, faixa in aliquotas_faixas.items():
        delta = max(saldo - faixa, 0)
        imposto += delta * aliquota
        saldo = min(saldo, faixa)
    inss = prolabore * 0.11
    return imposto + 12 * (contador + inss)


In [20]:
def resumo_pj(faturamento):
    custo_anual = custos_simples_anexo_iii(faturamento)
    return faturamento / 12, (faturamento - custo_anual) / 12, faturamento, faturamento - custo_anual



In [21]:
for faturamento in faturamento_bruto:
    print(*map(lambda v: f'{v:<15,.2f}', resumo_pj(faturamento)), sep=' - ')

2,133.30        - 1,795.31        - 25,599.65       - 21,543.67      
3,167.12        - 2,767.09        - 38,005.39       - 33,205.06      
4,202.85        - 3,740.68        - 50,434.25       - 44,888.20      
5,226.51        - 4,702.92        - 62,718.18       - 56,435.09      
9,337.04        - 8,566.82        - 112,044.53      - 102,801.86     
18,674.08       - 17,152.58       - 224,088.93      - 205,830.97     
28,011.11       - 25,443.87       - 336,133.33      - 305,326.40     
93,370.38       - 81,191.12       - 1,120,444.53    - 974,293.41     
933,703.71      - 671,841.49      - 11,204,444.53   - 8,062,097.84   


In [23]:
for salario, faturamento in zip(salario_anual_bruto, faturamento_bruto):
    print(f'Salário Mensal {salario:,.2f}')
    clt = resumo_empregado(salario)[2]
    print(f'CLT: R$ {clt :<15,.2f}', sep='')
    pj = resumo_pj(faturamento)[2]
    print(f'PJ: R$ {pj :<15,.2f}')
    print(f'DIFF: R${pj - clt :<15,.2f}')
    print()


Salário Mensal 1,903.98
CLT: R$ 25,190.67      
PJ: R$ 25,599.65      
DIFF: R$408.97         

Salário Mensal 2,826.66
CLT: R$ 36,653.29      
PJ: R$ 38,005.39      
DIFF: R$1,352.09       

Salário Mensal 3,751.06
CLT: R$ 47,324.32      
PJ: R$ 50,434.25      
DIFF: R$3,109.94       

Salário Mensal 4,664.68
CLT: R$ 57,195.13      
PJ: R$ 62,718.18      
DIFF: R$5,523.05       

Salário Mensal 8,333.34
CLT: R$ 95,016.21      
PJ: R$ 112,044.53     
DIFF: R$17,028.33      

Salário Mensal 16,666.67
CLT: R$ 181,071.73     
PJ: R$ 224,088.93     
DIFF: R$43,017.20      

Salário Mensal 25,000.00
CLT: R$ 267,127.25     
PJ: R$ 336,133.33     
DIFF: R$69,006.08      

Salário Mensal 83,333.34
CLT: R$ 869,516.21     
PJ: R$ 1,120,444.53   
DIFF: R$250,928.33     

Salário Mensal 833,333.34
CLT: R$ 8,614,516.21   
PJ: R$ 11,204,444.53  
DIFF: R$2,589,928.33   

